## Multi-tube design example for orthogonal systems (advanced) 

Design small conditional RNAs (scRNAs) for conditional Dicer substrate formation (Hochrein et al., J Am Chem Soc, 2013; see Figure 3). 

See the accompanying design specification (PDF files). See also the LaTeX spec file that you can edit to make your own design specs in a standardized format. 

This is a 2-step reaction. To design N orthogonal scRNA systems, there are 3 elementary step tubes per system plus 1 global crosstalk tube. 

Target test tubes: 
- (Step 0: Reactants) x N orthogonal systems 
- (Step 1: Intermediates) x N orthogonal systems
- (Step 2: Products) x N orthogonal systems
- Global crosstalk 

Material: RNA    
Temperature: 23 C

In [3]:
# Import Python NUPACK module
from nupack import *

In [4]:
# Define physical model
my_model = Model(material='rna', celsius=37)

# Set number of orthogonal systems
systems = 2 

# Initialize working variables
crosstalk_targets = {} 
crosstalk_excludes = [] 
step_tubes = [] 

# Loop over orthogonal systems
for i in range(systems):
    
    # Define sequence domains
    a = Domain('N6', name=['a', i])
    c = Domain('N8', name=['c', i])
    b = Domain('N4', name=['b', i])
    w = Domain('N2', name=['w', i])
    y = Domain('N4', name=['y', i])
    x = Domain('N12',name=['x', i])
    z = Domain('N3', name=['z', i])
    s = Domain('N5', name=['s', i])
    
    # Define strands containing these domains
    sC_out  = TargetStrand([w, x, y, s], name=['sC_out', i])
    sA      = TargetStrand([~c, ~b, ~a, ~z, ~y], name=['sA', i])
    sA_toe  = TargetStrand([~c], name=['sA_toe', i])
    sC      = TargetStrand([w, x, y, s, ~a, ~z, ~y, ~x, ~w], name=['sC', i])
    sC_loop = TargetStrand([s, ~a, ~z], name=['sC_loop', i])
    sB      = TargetStrand([x, y, z, a, b], name=['sB', i])
    sX      = TargetStrand([a, b, c], name=['sX', i])
    
    # Define target complexes
    C      = TargetComplex([sC], 'D2 D12 D4( U5 U6 U3 )', name=['C', i])
    B      = TargetComplex([sB], 'U12 U4 U3 U6 U4', name=['B', i])
    C_loop = TargetComplex([sC_loop], 'U14', name=['C_loop', i])
    AB    = TargetComplex([sA, sB], 'U8 D4 D6 D3 D4(+ U12)', name=['AB', i])
    X      = TargetComplex([sX], 'U18', name=['X', i])
    XA    = TargetComplex([sX, sA], 'D6 D4 D8(+) U3 U4', name=['XA', i])
    C_out  = TargetComplex([sC_out], 'U23', name=['C_out', i])
    BC    = TargetComplex([sB, sC], 'D12 D4 D3 D6 (U4 + U2 U12 U4 U5) U2', name=['BC', i])
    A_toe  = TargetComplex([sA_toe], 'U8', name=['A_toe', i])
    
    # Define 3 elementary step tubes for each orthogonal system
    Step_0 = TargetTube({C: 1e-08, X: 1e-08, AB: 1e-08}, 
        off_targets=SetSpec(max_size=2, include=[[sA], BC], exclude=[XA]), name=['Step_0', i])  
    Step_1 = TargetTube({XA: 1e-08, B: 1e-08}, 
        off_targets=SetSpec(max_size=2, include=[X, AB]), name=['Step_1', i])  
    Step_2 = TargetTube({BC: 1e-08}, 
        off_targets=SetSpec(max_size=2, include=[B, C]), name=['Step_2', i])
    step_tubes += [Step_0, Step_1, Step_2]
    
    # Store contributions to global crosstalk tube
    crosstalk_targets.update({
        AB: 1e-08,
        C: 1e-08,
        X: 1e-08,
        B: 1e-08,
        C_out: 1e-08,
        C_loop: 1e-08,
        A_toe: 1e-08,
    })   
    crosstalk_excludes += [XA, BC, [sX, sA_toe], [sB, sC_loop]]
    
# Define 1 global crosstalk tube 
crosstalk_tube = TargetTube(crosstalk_targets, 
                 off_targets=SetSpec(max_size=2, exclude=crosstalk_excludes), name='Crosstalk')

# Define tube set
my_tubes = step_tubes + [crosstalk_tube]

# Define defect weight of N for global crosstalk tube 
# (maintain balance between elementary step tubes and global crosstalk tube as number of 
# orthogonal systems increases) 
my_weights = Weights(my_tubes)
my_weights[:,:,:,crosstalk_tube] *= systems

# Set a stop condition of 5% 
# Set seed for random number generation to get a reproducible result for this demo
my_options = DesignOptions(f_stop=0.05, seed=93)

# Define and run the test tube design job
my_design = tube_design(tubes=my_tubes, defect_weights=my_weights, model=my_model, options=my_options)
my_results = my_design.run(trials=1)[0]

# Display the design results
my_results

Domain,Sequence
a[0],CACGAU
a[0]*,AUCGUG
a[1],AACACU
a[1]*,AGUGUU
b[0],UUAC
b[0]*,GUAA
b[1],CCGG
b[1]*,CCGG
c[0],GCUGCACU
c[0]*,AGUGCAGC
